In [1]:
%matplotlib notebook

In [2]:
#Dependencies 
import numpy as np
import pandas as pd
import requests
import time
import json
from us import states
from scipy.stats import linregress
from matplotlib import pyplot as plt
from glob import glob


ModuleNotFoundError: No module named 'us'

In [3]:
##### Covid data from CDC API starting here:
url = "https://data.cdc.gov/resource/vbim-akqf.json"
print(requests.get(url))
print(requests.get(url).json())

url = "https://data.cdc.gov/resource/vbim-akqf.json"
response = requests.get(url)
response_json = response.json()
print(json.dumps(response_json, indent=4, sort_keys=True))

<Response [200]>
[{'cdc_report_dt': '2020-07-03T00:00:00.000', 'current_status': 'Laboratory-confirmed case', 'sex': 'Female', 'age_group': '0 - 9 Years', 'race_ethnicity_combined': 'American Indian/Alaska Native, Non-Hispanic', 'hosp_yn': 'Missing', 'icu_yn': 'Missing', 'death_yn': 'Missing', 'medcond_yn': 'Missing'}, {'cdc_report_dt': '2020-05-27T00:00:00.000', 'current_status': 'Laboratory-confirmed case', 'sex': 'Female', 'age_group': '0 - 9 Years', 'race_ethnicity_combined': 'American Indian/Alaska Native, Non-Hispanic', 'hosp_yn': 'No', 'icu_yn': 'Missing', 'death_yn': 'No', 'medcond_yn': 'Missing'}, {'cdc_report_dt': '2020-06-06T00:00:00.000', 'current_status': 'Laboratory-confirmed case', 'sex': 'Female', 'age_group': '0 - 9 Years', 'race_ethnicity_combined': 'American Indian/Alaska Native, Non-Hispanic', 'hosp_yn': 'Missing', 'icu_yn': 'Missing', 'death_yn': 'Missing', 'medcond_yn': 'Missing'}, {'cdc_report_dt': '2020-06-28T00:00:00.000', 'current_status': 'Laboratory-confirme

[
    {
        "age_group": "0 - 9 Years",
        "cdc_report_dt": "2020-07-03T00:00:00.000",
        "current_status": "Laboratory-confirmed case",
        "death_yn": "Missing",
        "hosp_yn": "Missing",
        "icu_yn": "Missing",
        "medcond_yn": "Missing",
        "race_ethnicity_combined": "American Indian/Alaska Native, Non-Hispanic",
        "sex": "Female"
    },
    {
        "age_group": "0 - 9 Years",
        "cdc_report_dt": "2020-05-27T00:00:00.000",
        "current_status": "Laboratory-confirmed case",
        "death_yn": "No",
        "hosp_yn": "No",
        "icu_yn": "Missing",
        "medcond_yn": "Missing",
        "race_ethnicity_combined": "American Indian/Alaska Native, Non-Hispanic",
        "sex": "Female"
    },
    {
        "age_group": "0 - 9 Years",
        "cdc_report_dt": "2020-06-06T00:00:00.000",
        "current_status": "Laboratory-confirmed case",
        "death_yn": "Missing",
        "hosp_yn": "Missing",
        "icu_yn": "Missing",

In [4]:
cases_df =  pd.DataFrame({
    "State": ["Georgia", "California", "Illinois", "Texas", "New York"],
    "Cases": [175052, 466550, 174973, 394265, 225850]})
cases_df

,State,Cases
0,Georgia,175052
1,California,466550
2,Illinois,174973
3,Texas,394265
4,New York,225850


In [14]:
##### Jayden and Max working on the airline data merge here:

dataframes = []

#Max's work here:
df0 = pd.read_csv("/Users/bartholdy9000/upenn_hmwrk_max/project-airlines/project_data/airline data/airlineCSV/TFMSC_Report_2016.csv")
df1 = pd.read_csv("/Users/bartholdy9000/upenn_hmwrk_max/project-airlines/project_data/airline data/airlineCSV/TFMSC_Report_2017.csv")
df2 = pd.read_csv("/Users/bartholdy9000/upenn_hmwrk_max/project-airlines/project_data/airline data/airlineCSV/TFMSC_Report_2018.csv")
df3 = pd.read_csv("/Users/bartholdy9000/upenn_hmwrk_max/project-airlines/project_data/airline data/airlineCSV/TFMSC_Report_2019.csv")
df4 = pd.read_csv("/Users/bartholdy9000/upenn_hmwrk_max/project-airlines/project_data/airline data/airlineCSV/TFMSC_Report_2020.csv")

filenames = ["/Users/bartholdy9000/upenn_hmwrk_max/project-airlines/project_data/airline_data/airlineCSV/TFMSC_Report_2016.csv, 
             /Users/bartholdy9000/upenn_hmwrk_max/project-airlines/project_data/airline data/airlineCSV/TFMSC_Report_2017.csv, 
             /Users/bartholdy9000/upenn_hmwrk_max/project-airlines/project_data/airline data/airlineCSV/TFMSC_Report_2018.csv, 
             /Users/bartholdy9000/upenn_hmwrk_max/project-airlines/project_data/airline data/airlineCSV/TFMSC_Report_2019.csv, 
             /Users/bartholdy9000/upenn_hmwrk_max/project-airlines/project_data/airline data/airlineCSV/TFMSC_Report_2020.csv"
            ]

for f in filenames: dataframes.append(pd.read_csv(f))




SyntaxError: EOL while scanning string literal (<ipython-input-14-68f36c34c08f>, line 12)

In [ ]:
########### Dave's code below for 2017-18 flu ################
# import and clean data from CDC FluView website csv download

In [ ]:
# define data file
flu_state = "../project_data/flu_data/State_ILINet.csv"

In [ ]:
# read data file
raw_state_flu_2017 = pd.read_csv(flu_state)

In [ ]:
raw_state_flu_2017.head(15)

In [ ]:
#check data types
temp_state_flu_2017.dtypes

In [ ]:
# collect a list of all columns within the DataFrame
raw_state_flu_2017.columns

In [ ]:
# extract required columns to new DataFrame
temp_state_flu_2017 = raw_state_flu_2017[['REGION', 'YEAR', 'WEEK', 'ILITOTAL']]
temp_state_flu_2017.head()

In [ ]:
# drop rows with "X" in ILITOTAL column
temp_state_flu_2017 = temp_state_flu_2017[temp_state_flu_2017.ILITOTAL != 'X']
temp_state_flu_2017.head(15)

In [ ]:
# identify incomplete rows
temp_state_flu_2017.count()

In [ ]:
#check data types
temp_state_flu_2017.dtypes

In [ ]:
# use pd.to_numeric() method to convert the datatype of the ILITOTAL column
temp_state_flu_2017['ILITOTAL'] = pd.to_numeric(temp_state_flu_2017['ILITOTAL'])
temp_state_flu_2017['ILITOTAL'].dtype

In [ ]:
# rename the columns     
clean_state_flu_2017 = temp_state_flu_2017.rename(columns={"REGION":"State",
                                                           "YEAR":"Year",
                                                           "WEEK":"Week",
                                                           "ILITOTAL":"ILI Cases"})
clean_state_flu_2017.head()

In [ ]:
# determine the total number of ILI Cases
total_cases = clean_state_flu_2017["ILI Cases"].sum()
total_cases

In [ ]:
# determine the total of all ILI Cases by week
week_count = clean_state_flu_2017.groupby(['Year', 'Week']).sum()['ILI Cases']
week_count

In [ ]:
# count how many ILI cases have occured within each state
state_count = clean_state_flu_2017.groupby('State').sum()['ILI Cases']
state_count.head()

In [ ]:
# determine the total of all ILI Cases by week per state
state_week_count = clean_state_flu_2017.groupby(['State', 'Year', 'Week']).sum()['ILI Cases']
state_week_count.head(15)

In [ ]:
# Create a bar chart
y_axis = week_count 
x_axis = ['Year','Week']

plt.bar(x_axis, y_axis, color='lightblue', alpha=0.5, align="center")

# Tell matplotlib where we would like to place each of our x axis headers
tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, ['Week'])

# Sets the x limits of the current chart
plt.xlim(0, len(x_axis))

# Sets the y limits of the current chart
plt.ylim(0, max(week_count)+1000)

# Give our chart some labels and a tile
plt.title("2017-17 USA Influenza Epidemic")
plt.xlabel("Month")
plt.ylabel("Number of Influenza-Like-Infections \(ILI\)")